SHA 256 algorithm


https://qvault.io/cryptography/how-sha-2-works-step-by-step-sha-256/

https://www.researchgate.net/publication/259508311_A_compact_FPGA-based_processor_for_the_Secure_Hash_Algorithm_SHA-256 Outdated

https://github.com/xmrig/xmrig-proxy/wiki/API Seems like we are only able to use XMRig API to GET statistics about mining performance

As a result, may have to look elsewhere to implement custom algorithm.

https://xmrig.com/docs/miner/config/api#api-id

https://www.youtube.com/watch?v=ZhnJ1bkIWWk

https://medium.com/hackergirl/how-to-calculate-the-hash-of-a-block-in-bitcoin-8f6aebb0dc6d

https://analysis.null.place/how-do-the-bitcoin-mining-algorithms-work/#form4

Presentation:

https://analysis.null.place/how-do-the-bitcoin-mining-algorithms-work/#heading_block_hashing_in_detail

https://www.blockchain.com/btc/block/0

https://www.blockchain.com/explorer?view=btc

https://coinguides.org/x16r-algorithm-coins/

In [1]:
current_btc = 38317.74
JPYUSD = 115.47
print('BTC Price USD:', current_btc)
print('BTC Price Yen:', round(current_btc * JPYUSD, 2))

BTC Price USD: 38317.74
BTC Price Yen: 4424549.44


# Implementing SHA 256

https://qvault.io/cryptography/how-sha-2-works-step-by-step-sha-256/

In [2]:
# prints out binary input, working
def printb(string):
    for i in range(len(string)):
        if i > 0:
            if i%8 == 0:
                print(" ", end='')
                if i%64 == 0:
                    print("\n", end='')
        print(string[i], end='')
printb('0000000000000000')

00000000 00000000

In [3]:
# Step 0, convert string to binary
def convert_to_binary(test_str):
    return ''.join(format(ord(i), '08b') for i in test_str)
len(convert_to_binary('hello world'))

88

In [4]:
def int_to_bin(num, bits=32):
    # big endian
    # first separate into groups
    if bits == 64:
        num = '{:064b}'.format(num)
    elif bits == 32:
        num = '{:032b}'.format(num)
    return num
int_to_bin(88)

'00000000000000000000000001011000'

In [5]:
# Step 1, preprocessing
def preprocessing(binary_str):
    input_length = len(binary_str)
    binary_str += "1"
    for i in range(len(binary_str), 448):
        binary_str += '0'
    return binary_str + int_to_bin(input_length, bits=64)
printb(preprocessing(convert_to_binary('hello world')))

01101000 01100101 01101100 01101100 01101111 00100000 01110111 01101111 
01110010 01101100 01100100 10000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 
00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011000

In [6]:
# step 6 helper functions

# done, tested
def hex_to_bin(hex_val):
    return int_to_bin(int(hex_val, 16))



In [7]:
# step 2, initialize hash values

h0 = hex_to_bin("0x6a09e667")
h1 = hex_to_bin("0xbb67ae85")
h2 = hex_to_bin("0x3c6ef372")
h3 = hex_to_bin("0xa54ff53a")
h4 = hex_to_bin("0x510e527f")
h5 = hex_to_bin("0x9b05688c")
h6 = hex_to_bin("0x1f83d9ab")
h7 = hex_to_bin("0x5be0cd19")

h = [h0, h1, h2, h3, h4, h5, h6, h7]

In [8]:
# step 3, initialize more constants
constants = """0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5
0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174
0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da
0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967
0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85
0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070
0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3
0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2"""
constants = constants.replace("\n", " ")
k = constants.split(' ')
k = [hex_to_bin(i) for i in k]
print(k)

['01000010100010100010111110011000', '01110001001101110100010010010001', '10110101110000001111101111001111', '11101001101101011101101110100101', '00111001010101101100001001011011', '01011001111100010001000111110001', '10010010001111111000001010100100', '10101011000111000101111011010101', '11011000000001111010101010011000', '00010010100000110101101100000001', '00100100001100011000010110111110', '01010101000011000111110111000011', '01110010101111100101110101110100', '10000000110111101011000111111110', '10011011110111000000011010100111', '11000001100110111111000101110100', '11100100100110110110100111000001', '11101111101111100100011110000110', '00001111110000011001110111000110', '00100100000011001010000111001100', '00101101111010010010110001101111', '01001010011101001000010010101010', '01011100101100001010100111011100', '01110110111110011000100011011010', '10011000001111100101000101010010', '10101000001100011100011001101101', '10110000000000110010011111001000', '10111111010110010111111111

In [9]:
# step 4, no work necessary until scaling

In [10]:
# step 5 helper functions

# done, tested
def rightrotate(string, n):
    old_front = string[:-n]
    old_back = string[-n:]
    return old_back + old_front

def rightshift(string, n):
    old_front = string[:-n]
    old_back = '0' * n
    return old_back + old_front

def xor(string1, string2):
    assert len(string1) == len(string2)
    result = ''
    for i in range(len(string1)):
        if string1[i] == string2[i]:
            result += '0'
        else:
            result += '1'
    return result

def bin_and(string1, string2):
    assert len(string1) == len(string2)
    result = ''
    for i in range(len(string1)):
        if string1[i] == '1' and string2[i] == '1':
            result += '1'
        else:
            result += '0'
    return result

def bin_not(string):
    result = ''
    for i in range(len(string)):
        if string[i] == "1":
            result += '0'
        else:
            result += '1'
    return result

def bin_add(strings):
    total = 0
    for string in strings:
        total += to_decimal(string)
    return int_to_bin(total % (2**32))

def add(strings):
    return bin_add(strings)
    

def to_decimal(binary_str):
    return int(binary_str, 2)

In [11]:
# step 5, create message schedule (done, tested)
def message_schedule(input_data):
    w = []
    for i in range(16):
        start_index = 32 * i
        end_index = 32 * (i + 1)
        w.append(input_data[start_index:end_index])
    for i in range(48):
        w.append('00000000000000000000000000000000')
        
    for i in range(16, 64):
        s0 = xor(xor(rightrotate(w[i-15], 7), rightrotate(w[i-15], 18)), rightshift(w[i-15], 3))
        s1 = xor(xor(rightrotate(w[i-2], 17), rightrotate(w[i-2], 19)), rightshift(w[i-2], 10))
        w[i] = bin_add([w[i-16], s0, w[i-7], s1])    
        
    return w

In [12]:
# step 6, compression (done, tested)
def compression(w):
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    f = h5
    g = h6
    h = h7

    for i in range(64):
        s1 = xor(xor(rightrotate(e, 6), rightrotate(e, 11)), rightrotate(e, 25))
        ch = xor(bin_and(e, f), bin_and(bin_not(e), g))
        temp1 = bin_add([h, s1, ch, k[i], w[i]])
        s0 = xor(xor(rightrotate(a, 2), rightrotate(a, 13)), rightrotate(a, 22))
        maj = xor(xor(bin_and(a, b), bin_and(a, c)), bin_and(b, c))
        temp2 = bin_add([s0, maj])
        h = g
        g = f
        f = e
        e = add([d, temp1])
        d = c
        c = b
        b = a
        a = add([temp1, temp2])
    return [a, b, c, d, e, f, g, h]

In [13]:
# step 7, modify final values

def mod_final(h, letters):

    return [add([h[i], letters[i]]) for i in range(8)]

In [14]:
# step 8 helper functions
def bin_to_hex(string):
    without_leading_zeroes = hex(int(string, 2))[2:]
    return ('0' * (8 - len(without_leading_zeroes))) + without_leading_zeroes

In [15]:
# step 8, concatenate final hash
def concat(h):
    result = ''
    for i in h:
        result += bin_to_hex(i)
    return result

In [16]:
def sha256(string):
    
    
    h0 = hex_to_bin("0x6a09e667")
    h1 = hex_to_bin("0xbb67ae85")
    h2 = hex_to_bin("0x3c6ef372")
    h3 = hex_to_bin("0xa54ff53a")
    h4 = hex_to_bin("0x510e527f")
    h5 = hex_to_bin("0x9b05688c")
    h6 = hex_to_bin("0x1f83d9ab")
    h7 = hex_to_bin("0x5be0cd19")
    h = [h0, h1, h2, h3, h4, h5, h6, h7]
    
    constants = """0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5
0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174
0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da
0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967
0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85
0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070
0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3
0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2"""
    constants = constants.replace("\n", " ")
    constants = constants.replace("\t", "")
    k = constants.split(' ')
    k = [hex_to_bin(i) for i in k]
    
    input_str = preprocessing(convert_to_binary(string))
    w = message_schedule(input_str)
    final_hashes = mod_final(h, compression(w))
    return concat(final_hashes)

sha256('hello worldsss          s')

'3affd5adaccf2f74747d59e680808074ed3bd20daaf51cfd2a92609859456c8c'

# Using the algorithm for mining

In [17]:
# Sample SHA 256 encoding
import hashlib
a_string = 'hello world'
hashed_string = hashlib.sha256(a_string.encode('utf-8')).hexdigest()
print('Original String:', a_string)
print('Hash of string:', hashed_string)
print('Decimal value of hash:', int(hashed_string, 16))

Original String: hello world
Hash of string: b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9
Decimal value of hash: 83814198383102558219731078260892729932246618004265700685467928187377105751529


In [18]:
# Converts hex val to little endian
def to_little(val):
    if val[:2] == '0x': # handles formatting of hex()
        val = val[2:]
    little_hex = bytearray.fromhex(val)
    little_hex.reverse()

    str_little = ''.join(format(x, '02x') for x in little_hex)

    return str_little

In [19]:
# Converts and int to a little endian 4 byte hex value
# Used for version and nonce
def int_to_hex(num):
    num = '{:08x}'.format(num)
    return to_little(num)
print(int_to_hex(1))

01000000


In [20]:
# Generates the previous hash portion of the block header
def generate_prev_hash(test=True):
    if test:
        return '0000000000000000000000000000000000000000000000000000000000000000' # value for 1st bitcoin mined

In [21]:
#Generates the merkle root portion of the block header
def generate_merkle_root(test=True):
    if test:
        return to_little('4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b') # value for 1st bitcoin mined

In [22]:
# Generates the timestamp portion of block header
def generate_timestamp(test=True):
    if test:
        return to_little('495fab29') # hexadecimal timestamp of 1st bitcoin mined

In [23]:
# Generates the bits portion of the block header
def generate_bits(test=True):
    if test:
        return to_little('1d00ffff')

In [24]:
# Combines functions to create a the block header excluding the nonce
def generate_prefix():
    return int_to_hex(1) + generate_prev_hash() + generate_merkle_root() + generate_timestamp() + generate_bits()
generate_prefix()

'0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d'

In [34]:
# def sha256(string):
#     return hashlib.sha256(bytes.fromhex(string)).hexdigest()
help(hashlib.sha256)

Help on built-in function openssl_sha256 in module _hashlib:

openssl_sha256(string=b'', *, usedforsecurity=True)
    Returns a sha256 hash object; optionally initialized with a string



In [31]:
# Test to make sure header is constructed correctly
string = generate_prefix() + int_to_hex(2083236893)
string == '0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c'

True

In [32]:
def test_header(header):
    return to_little(sha256(sha256(header)))

In [33]:
test_header(string)

'a186ed52df3b883aef5809300dc0922c507043a5779f276913b842e53086297a'

In [28]:
# Iterates through nonce values until one produces a hash that is lower than the target
def find_nonce(prefix, target):
    i = 2083236892 # starting with a nonce close to the actual value
    header = ""
    while True:
        # status check
        if i %1000 == 0:
            print(i)
        
        header = prefix + int_to_hex(i)
        attempt = test_header(header)
        if attempt < target:
            return attempt, header, i
        i += 1
target = '00000000ffff0000000000000000000000000000000000000000000000000000'
find_nonce(generate_prefix(), target)

2083237000


KeyboardInterrupt: 